# Calculation of the resonant scattering
The following calculation is done using an approximation introduced by Stuhrmann

## Libraries

In [1]:
import pandas as pd
import os
import csv
import time

import numpy as np
from scipy import optimize
import scipy.integrate as integrate
import scipy.constants as con
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, LinearSegmentedColormap

## Job parameters

In [46]:
# give the metadada directory
data_dir = 'files\Data_for_ASAXS\Corrected_1D\\223_TestPorod'

# give the target file tyle:
target_file_type = '_c0_Sub.dat'

# enter the experimental Energies and X-ray attenuation coefficiants 
# in an exported ASCII file (True) or in this sctript (False)
enter_values_external = False

# ending for export file name containing energies and X-ray attenuatino coefficiants
export_file_name_ending = '_AttenuationCoefficiants.txt'
inport_file_name_ending = '_AttenuationCoefficiants_Complete.txt'

# Nessesary information to calculate the Resonant
list_FileEntries = ['Energy','dEnergy','f0','f-Real','f-Imagin']
list_Energy=[10000, 7000, 8004, 8239, 8304, 8324, 8330, 8973]
list_dEnergy=[1667, -1333, -329, -94, -29, -9, -3, 640]
list_f0=[28, 28, 28, 28, 28, 28, 28, 28]
list_fReal=[0.0, 0.0, -2.86, -4.11, -5.35, -6.63, -7.87, 0.0]
list_fImagin=[0.0, 0.0, 0.514, 0.489, 0.492, 0.541, 0.741, 0.0]

# Job
## Search the directory

In [47]:
# scan the directory
fileList = list()
for entry in os.scandir(data_dir):
    if entry.path.endswith(target_file_type) and entry.is_file():
        fileList.append(entry.path)

fileIterator = iter(fileList)
startFrame=pd.DataFrame()
for element in fileIterator: 
    startFrame = startFrame.append({'FileName':element.split(os.sep)[-1]},ignore_index=True)
if enter_values_external == True:
    for entry in list_FileEntries:
        startFrame[entry] = 'NaN'
    save_outPut_name = data_dir+os.sep+data_dir.split(os.sep)[-1]+export_file_name_ending
    startFrame.to_csv(save_outPut_name, sep='\t')
    print('File to enter values manualle exported to:',data_dir)
elif enter_values_external == False:
    startFrame['Energy']=list_Energy
    startFrame['dEnergy']=list_dEnergy
    startFrame['f0']=list_f0
    startFrame['f-Real']=list_fReal
    startFrame['f-Imagin']=list_fImagin
    print('Attenuation values completed from lists given above')
    pass
else:
    print('Enter True or False for >enter_values_external<')

Attenuation values completed from lists given above


## ReLoad the file containing the attenuation parameters

In [51]:
if enter_values_external == True:    
    #read the meta file
    path_toFile = data_dir+os.sep+data_dir.split(os.sep)[-1]+inport_file_name_ending 
    dF = pd.read_csv(path_toFile, sep='\t',index_col=0)
    print('Data is collected from:',data_dir)
elif enter_values_external==False:
    dF = startFrame
    print('Attenuation values completed from lists given above')
else:
    print('Enter True or False for >enter_values_external<')

Attenuation values completed from lists given above


In [52]:
dF

,FileName,Energy,dEnergy,f0,f-Real,f-Imagin
0,223_10000.0_eV_Tif_509_Norm_c0_Sub.dat,10000,1667,28,0.00,0.000
1,223_7000.0_eV_Tif_383_Norm_c0_Sub.dat,7000,-1333,28,0.00,0.000
2,223_8004.0_eV_Tif_401_Norm_c0_Sub.dat,8004,-329,28,-2.86,0.514
3,223_8239.0_eV_Tif_419_Norm_c0_Sub.dat,8239,-94,28,-4.11,0.489
4,223_8304.0_eV_Tif_437_Norm_c0_Sub.dat,8304,-29,28,-5.35,0.492
5,223_8324.0_eV_Tif_455_Norm_c0_Sub.dat,8324,-9,28,-6.63,0.541
6,223_8330.0_eV_Tif_473_Norm_c0_Sub.dat,8330,-3,28,-7.87,0.741
7,223_8973.0_eV_Tif_491_Norm_c0_Sub.dat,8973,640,28,0.00,0.000


In [48]:
startFrame

,FileName,Energy,dEnergy,f0,f-Real,f-Imagin
0,223_10000.0_eV_Tif_509_Norm_c0_Sub.dat,10000,1667,28,0.00,0.000
1,223_7000.0_eV_Tif_383_Norm_c0_Sub.dat,7000,-1333,28,0.00,0.000
2,223_8004.0_eV_Tif_401_Norm_c0_Sub.dat,8004,-329,28,-2.86,0.514
3,223_8239.0_eV_Tif_419_Norm_c0_Sub.dat,8239,-94,28,-4.11,0.489
4,223_8304.0_eV_Tif_437_Norm_c0_Sub.dat,8304,-29,28,-5.35,0.492
5,223_8324.0_eV_Tif_455_Norm_c0_Sub.dat,8324,-9,28,-6.63,0.541
6,223_8330.0_eV_Tif_473_Norm_c0_Sub.dat,8330,-3,28,-7.87,0.741
7,223_8973.0_eV_Tif_491_Norm_c0_Sub.dat,8973,640,28,0.00,0.000
